In [1]:
import os,time
import dask
from dask.distributed import Client
import xarray

In [2]:
client=Client(scheduler_file='../scheduler.json')
print(client)

<Client: 'tcp://10.128.1.185:6192' processes=7 threads=112, memory=130.39 GiB>


In [3]:
datasets=xarray.open_mfdataset(
    '/scratch/shaima0d/test_data/*.nc',
engine='netcdf4',
chunks={'time':50})
                        

import pandas as pd
years=list()
for i in range(len(datasets.time)):
    years.append(pd.to_datetime(datasets.time.data[i]).year)

In [4]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ytmp,dtmp=zip(*datasets.groupby('time.year'))

In [5]:
print(len(ytmp))
print(ytmp)

21
(1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965)


In [6]:
!rm -rf /scratch/shaima0d/results/*

In [7]:
paths=list()
for y in range(len(dtmp)):
    paths.append('/scratch/shaima0d/results/tmp%d.nc'%ytmp[y])

In [8]:
def writer(path,D):
    D.load()
    return D.to_netcdf(path=path,
    engine='netcdf4',
    format='NETCDF4_CLASSIC',
               )


In [9]:
futures=list()
for i in range(len(dtmp)):
    futures.append(client.submit(writer,paths[i],dtmp[i]))

In [10]:
for future in futures:
    print(future.result())

KeyError: 'data'